In [1]:
import pandas as pd
import requests
import numpy as np
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import pprint
from bs4 import BeautifulSoup
import re
import json 
import concurrent.futures
import time
import itertools



pp = pprint.PrettyPrinter()


retry_strategy = Retry(
    total=9,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    method_whitelist=["HEAD", "GET", "OPTIONS"]
)
adapter = HTTPAdapter(max_retries=retry_strategy)

client = requests.Session()
client.mount("https://", adapter)
client.mount("http://", adapter)



In [21]:
headers = {'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"}
base_url = 'https://www.booking.com/'
label = 'gen173nr-1DCAEoggJCAlhYSDNYBGiTAYgBAZgBLsIBCndpbmRvd3MgMTDIAQzYAQPoAQGSAgF5qAID'
sid ='9a8c5833856fc2a673c8768cd752f3d1'

## Creating Reference Tables

### country dataset creation

In [8]:
# Reference table creation
# we created this function to get the entire list of countries 

#df_countries = pd.DataFrame()
def extract_country(df_countries):
    dest_id_range = range(0,250)#range(245,500)
    for dest_id in dest_id_range:
        url = (
        "https://www.booking.com/searchresults.en-gb.html"
        "?aid=304142&sb=1"
        #"&checkin_year=2021&checkin_month=6&checkin_monthday=12"
        #"&checkout_year=2021&checkout_month=6&checkout_monthday=15"
        "&ac_langcode=en&dest_id={dest_id}&dest_type=country".format(
            label=label
            ,sid=sid
            ,dest_id=dest_id
            ))
        r = requests.get(url,headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        header_text = soup.find('div',{'class':'sr_header'}).get_text().strip() if soup.find('div',{'class':'sr_header'}) is not None else None
        #if a dest_id does exist, it will have some properties listed. Aka here we check that dest_id is valid.
        if header_text is not None:
            if ':' in soup.find('div',{'class':'sr_header'}).get_text().strip():
                country_name = header_text.split(':')[0]
                number_listings = np.int(re.sub('\D','',header_text.split(':')[1].strip()))
                df_countries = pd.concat([df_countries,pd.DataFrame({'dest_id':dest_id,'country_name':country_name,'number_listings':number_listings,'url_link':url},index=[0])])
    return df_countries

### city dataset creation

In [124]:

# function to create reference_table_city_names

def reference_table_creation_country_get_all_pages_urls(number_listings,dest_id_country):
    label = 'gen173nr-1DCAEoggJCAlhYSDNYBGiTAYgBAZgBLsIBCndpbmRvd3MgMTDIAQzYAQPoAQGSAgF5qAID'
    sid ='9a8c5833856fc2a673c8768cd752f3d1'
    listings_per_page = 25
    n_pages = int(np.floor(number_listings/listings_per_page))
    offsets = [x * listings_per_page for x in range(n_pages+1)] + [number_listings - 3]
    list_urls = []
    for offset in offsets:
        url = (
            "https://www.booking.com/searchresults.en-gb.html"
            "?aid=304142&sb=1"
            #"&checkin_year=2021&checkin_month=6&checkin_monthday=12"
            #"&checkout_year=2021&checkout_month=6&checkout_monthday=15"
            "&ac_langcode=en&dest_id={dest_id_country}&dest_type=country&rows={rows}&offset={offset}".format(
                label=label
                ,sid=sid
                ,dest_id_country=dest_id_country
                ,rows=25
                ,offset=offset
                ))
        list_urls.append(url)
    return list(set(list_urls))


#help functions
def reference_table_creation_get_city_name(soup):
    for _ in range(len(soup.find_all('input',type='hidden'))):
        if soup.find_all('input',type='hidden')[_].get('name')=='dest_id':
            #print(soup.find_all('input',type='hidden')[_].get('name'),soup.find_all('input',type='hidden')[_].get('value'))
            dest_id_city = soup.find_all('input',type='hidden')[_].get('value')
            break
    for _ in range(len(soup.find_all('input',type='hidden'))):
        if soup.find_all('input',type='hidden')[_].get('name')=='ssne':
            #print(soup.find_all('input',type='hidden')[_].get('name'),soup.find_all('input',type='hidden')[_].get('value'))
            city_name = soup.find_all('input',type='hidden')[_].get('value')
            break
    try:
        dest_id_city=dest_id_city
    except UnboundLocalError:
        dest_id_city=np.nan
    try:
        city_name=city_name 
    except UnboundLocalError:
        city_name=np.nan
        
    return dest_id_city,city_name


def reference_table_creation_create_city_record(tmp,dest_id_city,city_name):
    tmp ={
    'dest_id_city':dest_id_city
    ,'city_name':city_name
    ,'address_streetAddress': tmp['address']['streetAddress']
    ,'address_addressLocality':tmp['address']['addressLocality']
    ,'address_addressRegion':tmp['address']['addressRegion']
    ,'address_type':tmp['address']['@type']
    ,'address_postalCode':tmp['address']['postalCode']
    ,'address_addressCountry':tmp['address']['addressCountry']
    }
    return pd.DataFrame(tmp,index=[0])



def reference_table_generate_df_cities(number_listings,dest_id_country):
    
    df_cities = pd.DataFrame()
    label = 'gen173nr-1DCAEoggJCAlhYSDNYBGiTAYgBAZgBLsIBCndpbmRvd3MgMTDIAQzYAQPoAQGSAgF5qAID'
    sid ='9a8c5833856fc2a673c8768cd752f3d1'
    url = (
        "https://www.booking.com/searchresults.en-gb.html"
        "?aid=304142&sb=1"
        #"&checkin_year=2021&checkin_month=6&checkin_monthday=12"
        #"&checkout_year=2021&checkout_month=6&checkout_monthday=15"
        "&ac_langcode=en&dest_id={dest_id}&dest_type=country".format(
            label=label
            ,sid=sid
            ,dest_id=dest_id_country
            ))
    
    list_urls = reference_table_creation_country_get_all_pages_urls(number_listings,dest_id_country)
    
    def concurrency_get_single_urls(url):
        single_urls = []
        r = requests.get(url,headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        n_results = len(soup.find_all('a',{'class':'js-sr-hotel-link hotel_name_link url'}))
        
        for idx in range(n_results):
            href = (soup.find_all('a',{'class':'js-sr-hotel-link hotel_name_link url'})[idx].get('href').replace('\n',''))
            single_urls.append(base_url+href)
        return (single_urls)
    
    single_urls = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(concurrency_get_single_urls, list_urls)
        for result in results:
            single_urls.append(result)
            
    single_urls = (list(itertools.chain.from_iterable(single_urls)))
    
    print('Single url found: {}'.format(len(single_urls)))
    
    def concurrency_get_df_cities(single_url):
        #print(single_url)
        r = requests.get(single_url,headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        tmp = json.loads("".join(soup.find("script", {"type":"application/ld+json"}).contents))
        dest_id_city,city_name = reference_table_creation_get_city_name(soup)
        
        return reference_table_creation_create_city_record(tmp,dest_id_city,city_name)
    
    list_dfs = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(concurrency_get_df_cities, single_urls)
        for result in results:
            list_dfs.append(result)
    
    return list_dfs



In [3]:
# loading existing records

#df_cities = pd.read_csv('booking.com_reference_table_list_cities.csv')
#df_countries = pd.read_csv('booking.com_reference_table_list_countries.csv')

In [481]:
#create reference table
#collect all dest_id and country names
#per each country name,check all listings and get the dest_id - map it to city name

In [4]:
#df_cities = pd.DataFrame()


,dest_id,country_name,number_listings,url_link
5,6,Angola,30,https://www.booking.com/searchresults.en-gb.ht...


In [79]:
#df_cities.address_addressCountry.value_counts().head(60)

In [125]:
df_countries_shrinked = df_countries[~df_countries.country_name.isin(df_cities.address_addressCountry.value_counts().index.tolist())].copy()
#df_countries_shrinked = df_countries_shrinked[df_countries_shrinked.country_name=='Angola']
df_countries_shrinked = df_countries_shrinked[~df_countries_shrinked.dest_id.isin([41,46,69,50,179,182,245,53,180])]

df_countries_shrinked = df_countries_shrinked[(df_countries_shrinked.number_listings<550) & (df_countries_shrinked.number_listings>0)]
print('Info :\nNumber of countries to extract: {}'.format(df_countries_shrinked.shape[0]))
country = df_countries_shrinked[['dest_id','country_name','number_listings']].values.tolist()[:4]
country

Info :
Number of countries to extract: 5


[[139, 'Moldova', 471],
 [153, 'Nicaragua', 518],
 [160, 'Oman', 519],
 [188, 'Seychelles', 500]]

https://www.booking.com/searchresults.en-gb.html?aid=304142&sb=1&ac_langcode=en&dest_id=7&dest_type=country


In [126]:
def reference_table_generate_df_cities(number_listings,dest_id_country):
    
    df_cities = pd.DataFrame()
    label = 'gen173nr-1DCAEoggJCAlhYSDNYBGiTAYgBAZgBLsIBCndpbmRvd3MgMTDIAQzYAQPoAQGSAgF5qAID'
    sid ='9a8c5833856fc2a673c8768cd752f3d1'
    url = (
        "https://www.booking.com/searchresults.en-gb.html"
        "?aid=304142&sb=1"
        #"&checkin_year=2021&checkin_month=6&checkin_monthday=12"
        #"&checkout_year=2021&checkout_month=6&checkout_monthday=15"
        "&ac_langcode=en&dest_id={dest_id}&dest_type=country".format(
            label=label
            ,sid=sid
            ,dest_id=dest_id_country
            ))
    
    list_urls = reference_table_creation_country_get_all_pages_urls(number_listings,dest_id_country)
    
    def concurrency_get_single_urls(url):
        single_urls = []
        r = requests.get(url,headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        n_results = len(soup.find_all('a',{'class':'js-sr-hotel-link hotel_name_link url'}))
        
        for idx in range(n_results):
            href = (soup.find_all('a',{'class':'js-sr-hotel-link hotel_name_link url'})[idx].get('href').replace('\n',''))
            single_urls.append(base_url+href)
        return (single_urls)
    
    single_urls = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(concurrency_get_single_urls, list_urls)
        for result in results:
            single_urls.append(result)
            
    single_urls = (list(itertools.chain.from_iterable(single_urls)))
    
    print('Single url found: {}'.format(len(single_urls)))
    
    def concurrency_get_df_cities(single_url):
        #print(single_url)
        r = requests.get(single_url,headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        tmp = json.loads("".join(soup.find("script", {"type":"application/ld+json"}).contents))
        dest_id_city,city_name = reference_table_creation_get_city_name(soup)
        
        return reference_table_creation_create_city_record(tmp,dest_id_city,city_name)
    
    list_dfs = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(concurrency_get_df_cities, single_urls)
        for result in results:
            list_dfs.append(result)
    
    return list_dfs


In [ ]:
# original 

t1 = time.perf_counter()
data = []

def concurrent_execution(country):
    dest_id_country =  country[0]
    number_listings = country[2]
    print('Extracting data: {} ({}). Number Listings {}'.format(country[1],country[0],country[2]))
    
    print('------------------------------------------------------------------------------------------')
    return reference_table_generate_df_cities(number_listings,dest_id_country)



with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(concurrent_execution,country)    
    for result in results:
        data.append(result)


list_dfs = (list(itertools.chain.from_iterable(data)))
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')
#df_cities = pd.concat([df_cities,update_df_cities(data)])

Extracting data: Moldova (139). Number Listings 471
------------------------------------------------------------------------------------------
Extracting data: Nicaragua (153). Number Listings 518
------------------------------------------------------------------------------------------
Extracting data: Oman (160). Number Listings 519
------------------------------------------------------------------------------------------
Extracting data: Seychelles (188). Number Listings 500
------------------------------------------------------------------------------------------


In [119]:
#save new records
#df_countries.to_csv('booking.com_reference_table_list_countries.csv',index=False)
df_cities.to_csv('booking.com_reference_table_list_cities.csv',index=False)

In [120]:
#df_cities = pd.DataFrame()
for idx in range(len(list_dfs)):
    df_cities = pd.concat([df_cities,list_dfs[idx]])
#df_cities

In [87]:
df_cities.groupby(['address_addressCountry','city_name']).count()#.reset_index()['address_addressCountry'].value_counts().head(60)

dest_id_city  address_streetAddress  \
address_addressCountry city_name                                             
Afghanistan            Herāt                      1                      1   
                       Kabul                      5                      5   
                       Shīrpūr                    2                      2   
Algeria                'Aïn el Turk               1                      1   
                       Akbou                      1                      1   
...                                             ...                    ...   
Zimbabwe               Mutare                     4                      4   
                       Odzi                       1                      1   
                       Ruwa                       1                      1   
                       Strathmore                 1                      1   
                       Victoria Falls            30                     30   

                                       address_addressLocality  \
address_addressCountry city_name                                 
Afghanistan            Herāt                                 1   
                       Kabul                                 5   
                       Shīrpūr                               2   
Algeria                'Aïn el Turk                          1   
                       Akbou                                 1   
...                                                        ...   
Zimbabwe               Mutare                                4   
                       Odzi                                  1   
                       Ruwa                                  1   
                       Strathmore                            0   
                       Victoria Falls                       30   

                                       address_addressRegion  address_type  \
address_addressCountry city_name                                             
Afghanistan            Herāt                               0             1   
                       Kabul                               5             5   
                       Shīrpūr                             2             2   
Algeria                'Aïn el Turk                        1             1   
                       Akbou                               1             1   
...                                                      ...           ...   
Zimbabwe               Mutare                              0             4   
                       Odzi                                0             1   
                       Ruwa                                0             1   
                       Strathmore                          0             1   
                       Victoria Falls                      0            30   

                                       address_postalCode  
address_addressCountry city_name                           
Afghanistan            Herāt                            1  
                       Kabul                            5  
                       Shīrpūr                          0  
Algeria                'Aïn el Turk                     1  
                       Akbou                            1  
...                                                   ...  
Zimbabwe               Mutare                           0  
                       Odzi                             0  
                       Ruwa                             0  
                       Strathmore                       1  
                       Victoria Falls                   1  

[1244 rows x 6 columns]

In [115]:
df_cities.shape

(10803, 8)

In [103]:
len(set(df_cities['city_name'])),len(set(df_cities['address_addressCountry']))

(1402, 106)

----------------

In [ ]:
label = 'gen173nr-1DCAEoggJCAlhYSDNYBGiTAYgBAZgBLsIBCndpbmRvd3MgMTDIAQzYAQPoAQGSAgF5qAID'
sid ='9a8c5833856fc2a673c8768cd752f3d1'
url = (
    "https://www.booking.com/searchresults.en-gb.html"
    "?aid=304142&sb=1"
    #"&checkin_year=2021&checkin_month=6&checkin_monthday=12"
    #"&checkout_year=2021&checkout_month=6&checkout_monthday=15"
    "&ac_langcode=en&dest_id={dest_id}&dest_type=country".format(
        label=label
        ,sid=sid
        ,dest_id=7
        ))
print(url)